Let's try generating a DC-SBM properly and then do the PPR stuff again.  This way we don't have to worry about errors translating from the SBM.

10 communities w/ 100 nodes each, 1 one with 1,000

In [464]:
import numpy as np

In [465]:
# block membership matrix
Z = np.zeros((2000, 11), dtype=int)

sizes = np.append(np.array([100]*10), 1000)
C = len(sizes)
N = sizes.sum()

thresholds = np.zeros(len(sizes) + 1, dtype=int)
thresholds[1:] = sizes

for i in range(2, len(thresholds)):
    thresholds[i:] += sizes[i-2]

for i in range(C):
    Z[thresholds[i]:thresholds[i+1]][:,i] = 1

    
# mixing matrix
# average of 8 edges between the smaller off-diagonal classes
B = np.ones((C,C)) * 8
B[:,-1] *= 10
B[-1, :] *= 10

for i in range(C -1):
    B[i,i] = 3200
    
B[-1,-1] = 320000

# theta should sum to 1 for each block
theta = np.ones(N)
theta[:1000] *= 1e-2
theta[1000:] *= 1e-3

assert theta.sum() == C

# generate random graph
A = np.zeros((N,N), dtype=int)
rng = np.random.default_rng(7777778)
choices = rng.uniform(size=N*(N-1)//2)

# probabilities for each pair of nodes
theta = np.diag(theta)
p = (theta @ Z @ B @ Z.T @ theta)

# this sucks but yeet
A = np.zeros((N,N), dtype=int)

k = 0
for i in range(N):
    for j in range(i+1, N):
        if choices[k] < p[i,j]:
            A[i,j] = 1
            A[j,i] = 1
        k+=1

In [466]:
for i in range(C):
    print(A[thresholds[i]:thresholds[i+1]].sum() - B.sum(axis=0)[i])

88.0
-110.0
-92.0
-37.0
29.0
-12.0
-21.0
-49.0
48.0
47.0
-757.0


In [467]:
for i in range(C):
    print(
        A[thresholds[i]:thresholds[i+1], thresholds[i]:thresholds[i+1]].sum() - B[i,i])

86.0
-104.0
-102.0
-38.0
40.0
2.0
-24.0
-68.0
56.0
54.0
-746.0


In [468]:
B.sum(axis=0)

array([  3352.,   3352.,   3352.,   3352.,   3352.,   3352.,   3352.,
         3352.,   3352.,   3352., 320800.])

Ok I am satisfied.  This DC-SBM looks legit.

Compute blockwise PPR

In [469]:
# fix source node of PPR for now
u = 7
alpha = 0.3

pi = np.zeros(N)
pi[u] = 1

D = np.diag(B.sum(axis=0))
P = np.linalg.inv(D) @ B

alpha_pi_t = pi @ Z * alpha


inv_transform = np.linalg.inv(np.eye(C) - (1-alpha)*P)
p = alpha_pi_t @ inv_transform

# blockwise PPR vector
p

array([0.90456059, 0.00477533, 0.00477533, 0.00477533, 0.00477533,
       0.00477533, 0.00477533, 0.00477533, 0.00477533, 0.00477533,
       0.05246141])

In [397]:
from lassort.localassort import calculateRWRrange

# is this correct?
degree = A.sum(axis=0)
D = np.diag(1./degree, 0)
# construct transition matrix (row normalised adjacency matrix)
W = D @ A

pr = np.array([alpha])

pis, ti, it = calculateRWRrange(W, 100, pr, N)

In [398]:
ti[:100].sum()

0.0007084564687767997

In [399]:
ti[100:200].sum()

0.913508264314111

In [400]:
ti[200:300].sum()

0.0007069888864222242

In [401]:
ti[400:500].sum()

0.0011422256022391288

Ooookay...  I think this is right actually.

In [470]:
# compute expected  multiscale mixing
# egl - ar^2
ar2 = (B.sum(axis=0) / B.sum())**2

eggl = p * np.diag(B)/ B.sum(axis=0)


qmax = 1 - ar2.sum()
r_loc = (eggl.sum() - ar2.sum()) / qmax

r_loc

0.7597160126411407

In [436]:
from lassort import localAssortF
import networkx as nx

In [452]:
G = nx.convert_matrix.from_numpy_matrix(A)
E = nx.convert_matrix.to_pandas_edgelist(G).values[:,:2]


In [455]:
attr = []

for i in range(C - 1):
    attr.append(np.ones(100, dtype=int) * i)
    
attr.append(np.ones(1000, dtype=int)*10)

M = np.hstack(attr)
assortM, assortT, Z = localAssortF(E,M,pr=np.array([0.3]))
                                   
assortT

array([0.7992832 , 0.6635741 , 0.79227753, ..., 0.98362697, 0.9833658 ,
       0.96762478])

In [458]:
assortT[:1000].mean()

0.7563480046505685

This is close...

Let's try one in the big chonk

In [471]:
# fix source node of PPR for now
u = 1554
alpha = 0.3

pi = np.zeros(N)
pi[u] = 1

D = np.diag(B.sum(axis=0))
P = np.linalg.inv(D) @ B

alpha_pi_t = pi @ Z * alpha


inv_transform = np.linalg.inv(np.eye(C) - (1-alpha)*P)
p = alpha_pi_t @ inv_transform

# compute expected  multiscale mixing
# egl - ar^2
ar2 = (B.sum(axis=0) / B.sum())**2

eggl = p * np.diag(B)/ B.sum(axis=0)


qmax = 1 - ar2.sum()
r_loc = (eggl.sum() - ar2.sum()) / qmax

r_loc

0.984786876776065

In [472]:
assortT[1000:].mean()

0.9854791201628526

In [461]:
pi.shape

(2000,)

In [462]:
Z.shape

(2000,)